In [0]:
spark.conf.set(
    "fs.azure.account.key.projectadf1.dfs.core.windows.net",
    "CSiXNWhibZG0fqyBMd04Qn4++bAtOoS+HwZt5IWin/Pk9UrUimppMcGnXCzGUICGV/PCey4nuGzs+AStQ5MAfQ==")

## Read required processed data

In [0]:
appointments_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/appointments")
doctors_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/doctors")
billing_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/billing")

## Show schema to confirm column names

In [0]:
appointments_df.printSchema()
billing_df.printSchema()

root
 |-- appointment_id: integer (nullable = true)
 |-- appointment_timestamp: timestamp (nullable = true)
 |-- appointment_date: date (nullable = true)
 |-- appointment_year: string (nullable = true)
 |-- appointment_month: string (nullable = true)
 |-- appointment_day: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- doctor_id: integer (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)

root
 |-- invoice_id: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- amount: double (nullable = true)



## Transformation 1: Number of doctors in each specialization

In [0]:
# Transformation 1: Number of doctors in each specialization
from pyspark.sql.functions import count
doctors_per_specialization = doctors_df.groupBy("specialization").agg(
count("doctor_id").alias("number_of_doctors")
)
print("Number of doctors per specialization:")
doctors_per_specialization.show()

Number of doctors per specialization:
+--------------------+-----------------+
|      specialization|number_of_doctors|
+--------------------+-----------------+
|          cardiology|               23|
|  infectious disease|               37|
|        geriatrician|               20|
|    gastroenterology|               33|
|          psychiatry|               20|
|critical care med...|               15|
|  emergency medicine|               31|
|       ophthalmology|               25|
|      pulmonologists|               27|
|          nephrology|               26|
|     endocrinologist|               21|
|   otolaryngologists|               32|
|           neurology|               24|
|     family medicine|               19|
|          allergists|               28|
|          internists|               28|
|obstetric anesthe...|               24|
|         dermatology|               29|
|           radiology|               21|
|      anesthesiology|               25|
+------------------

## Transformation 2: Revenue per year by doctor and specialization


In [0]:
from pyspark.sql.functions import sum, col

# Join billing with appointments using patient_id to get doctor_id and year, then with doctors for specialization
revenue_by_doctor_year = billing_df.join(
    appointments_df.select("patient_id", "doctor_id", "appointment_year"),
    "patient_id",
    "inner"
).join(
    doctors_df.select("doctor_id", "doctor_name", "specialization"),
    "doctor_id",
    "inner"
).groupBy("doctor_id", "doctor_name", "specialization", "appointment_year").agg(
    sum("amount").alias("yearly_revenue")
)

print("Revenue by doctor and year:")
revenue_by_doctor_year.orderBy("appointment_year", "yearly_revenue", ascending=False).show()

Revenue by doctor and year:
+---------+-----------+--------------------+----------------+--------------+
|doctor_id|doctor_name|      specialization|appointment_year|yearly_revenue|
+---------+-----------+--------------------+----------------+--------------+
|      667|     Gloria|        geriatrician|            2023|     4035149.0|
|      801|     Brooks|         dermatology|            2023|     3436703.0|
|      452|      Minne|critical care med...|            2023|     3436703.0|
|      189|        Max|     family medicine|            2023|     3283643.0|
|      639|     Arabel|          pediatrics|            2023|     2618062.0|
|      262|       Maye|      pulmonologists|            2023|     2382142.0|
|      170|   Quintina|          oncologist|            2023|     2281671.0|
|      975|     Hettie|          allergists|            2023|     2173939.0|
|      997|   Georgina|obstetric anesthe...|            2023|     2037322.0|
|      692|     Nariko|          internists|    

## Transformation 3: Total revenue for each doctor by year


In [0]:
total_revenue_by_doctor_year = revenue_by_doctor_year.groupBy("doctor_id", "doctor_name", "specialization", "appointment_year").agg(
    sum("yearly_revenue").alias("total_yearly_revenue")
)

print("Total revenue per doctor by year:")
total_revenue_by_doctor_year.orderBy("appointment_year", "total_yearly_revenue", ascending=False).show()

Total revenue per doctor by year:
+---------+-----------+--------------------+----------------+--------------------+
|doctor_id|doctor_name|      specialization|appointment_year|total_yearly_revenue|
+---------+-----------+--------------------+----------------+--------------------+
|      667|     Gloria|        geriatrician|            2023|           4035149.0|
|      801|     Brooks|         dermatology|            2023|           3436703.0|
|      452|      Minne|critical care med...|            2023|           3436703.0|
|      189|        Max|     family medicine|            2023|           3283643.0|
|      639|     Arabel|          pediatrics|            2023|           2618062.0|
|      262|       Maye|      pulmonologists|            2023|           2382142.0|
|      170|   Quintina|          oncologist|            2023|           2281671.0|
|      975|     Hettie|          allergists|            2023|           2173939.0|
|      997|   Georgina|obstetric anesthe...|         

## Transformation 4: Yearly specilization count for each year

In [0]:
from pyspark.sql.functions import col, count

# Ensure year is integer
appointments_clean = appointments_df.withColumn(
    "year_int", col("appointment_year").cast("int")
)

# Join with doctors to bring specialization info
yearly_specialization_appointments = (
    appointments_clean.join(
        doctors_df.select("doctor_id", "specialization"),
        "doctor_id",
        "inner"
    )
    .groupBy("specialization", "year_int")
    .agg(count("appointment_id").alias("yearly_appointment_count"))
    .orderBy("year_int", "yearly_appointment_count", ascending=False)
)

print("Yearly appointment count per specialization:")
yearly_specialization_appointments.show()


Yearly appointment count per specialization:
+--------------------+--------+------------------------+
|      specialization|year_int|yearly_appointment_count|
+--------------------+--------+------------------------+
|     endocrinologist|    2023|                       7|
|          oncologist|    2023|                       6|
|       ophthalmology|    2023|                       6|
|          nephrology|    2023|                       5|
|          allergists|    2023|                       5|
|obstetric anesthe...|    2023|                       5|
|  infectious disease|    2023|                       5|
|          pediatrics|    2023|                       5|
|         dermatology|    2023|                       5|
|      anesthesiology|    2023|                       5|
|        geriatrician|    2023|                       5|
|          cardiology|    2023|                       4|
|critical care med...|    2023|                       4|
|  emergency medicine|    2023|            

In [0]:
from pyspark.sql.functions import min, max, avg

yearly_stats = yearly_specialization_appointments.agg(
    min("yearly_appointment_count").alias("min_count"),
    max("yearly_appointment_count").alias("max_count"),
    avg("yearly_appointment_count").alias("avg_count")
)

print("Range of yearly specialization appointment counts:")
yearly_stats.show()


Range of yearly specialization appointment counts:
+---------+---------+-----------------+
|min_count|max_count|        avg_count|
+---------+---------+-----------------+
|        1|        9|4.252631578947368|
+---------+---------+-----------------+



In [0]:
from pyspark.sql.functions import when

# Join with doctors to bring specialization info
specialization_demand_yearly = (
    yearly_specialization_appointments.groupBy("specialization")
    .agg(avg("yearly_appointment_count").alias("avg_yearly_appointments"))
    .withColumn(
        "specialization_demand_level",
        when(
            (col("avg_yearly_appointments") >= 5.5), "High Demand"
        )
        .when(
            (col("avg_yearly_appointments") >= 4.25) & (col("avg_yearly_appointments") < 5.5),
            "Medium Demand"
        )
        .otherwise("Low Demand")
    )
)


In [0]:
specialization_demand_yearly.display()

specialization,avg_yearly_appointments,specialization_demand_level
cardiology,3.5,Low Demand
infectious disease,6.25,High Demand
geriatrician,4.5,Medium Demand
gastroenterology,5.75,High Demand
psychiatry,3.0,Low Demand
critical care medicine,2.75,Low Demand
emergency medicine,5.0,Medium Demand
ophthalmology,5.25,Medium Demand
pulmonologists,5.0,Medium Demand
nephrology,3.75,Low Demand


## Transformation 5: Yearly revenue trend for each specialization


In [0]:
# Transformation 8: Yearly revenue trend for each specialization
specialization_revenue_trend = revenue_by_doctor_year.groupBy("specialization", "appointment_year").agg(
    sum("yearly_revenue").alias("total_yearly_revenue_specialization"),
    count("doctor_id").alias("number_of_doctors")
).withColumn(
    "avg_revenue_per_doctor", col("total_yearly_revenue_specialization") / col("number_of_doctors")
)

print("Specialization revenue trend:")
specialization_revenue_trend.orderBy("appointment_year", "total_yearly_revenue_specialization", ascending=False).show()

Specialization revenue trend:
+--------------------+----------------+-----------------------------------+-----------------+----------------------+
|      specialization|appointment_year|total_yearly_revenue_specialization|number_of_doctors|avg_revenue_per_doctor|
+--------------------+----------------+-----------------------------------+-----------------+----------------------+
|     endocrinologist|            2023|                          7830487.0|                7|             1118641.0|
|critical care med...|            2023|                          6256922.0|                3|    2085640.6666666667|
|obstetric anesthe...|            2023|                          5815321.0|                4|            1453830.25|
|        geriatrician|            2023|                          4844910.0|                2|             2422455.0|
|          pediatrics|            2023|                          4644340.0|                3|    1548113.3333333333|
|     family medicine|            

## Which specialization has the highest workload

In [0]:
from pyspark.sql.functions import sum

specialization_workload = (
    yearly_specialization_appointments
    .groupBy("specialization")
    .agg(sum("yearly_appointment_count").alias("total_appointments"))
    .orderBy("total_appointments", ascending=False)
)

specialization_workload.show()

+--------------------+------------------+
|      specialization|total_appointments|
+--------------------+------------------+
|  infectious disease|                25|
|          oncologist|                24|
|    gastroenterology|                23|
|         dermatology|                23|
|       ophthalmology|                21|
|   otolaryngologists|                21|
|          internists|                21|
|  emergency medicine|                20|
|      pulmonologists|                20|
|        geriatrician|                18|
|obstetric anesthe...|                17|
|           neurology|                16|
|          allergists|                16|
|          nephrology|                15|
|           radiology|                15|
|      anesthesiology|                15|
|          pediatrics|                15|
|          cardiology|                14|
|             surgery|                14|
|     endocrinologist|                13|
+--------------------+------------

In [0]:
from pyspark.sql.functions import when, col

specialization_workload_classified = (
    specialization_workload
    .withColumn(
        "workload_level",
        when(col("total_appointments") >= 22, "High Workload")
        .when((col("total_appointments") >= 17) & (col("total_appointments") < 22), "Medium Workload")
        .otherwise("Low Workload")
    )
    .orderBy("total_appointments", ascending=False)
)

print("Specialization Workload Classification:")
specialization_workload_classified.show(truncate=False)


Specialization Workload Classification:
+---------------------------+------------------+---------------+
|specialization             |total_appointments|workload_level |
+---------------------------+------------------+---------------+
|infectious disease         |25                |High Workload  |
|oncologist                 |24                |High Workload  |
|gastroenterology           |23                |High Workload  |
|dermatology                |23                |High Workload  |
|ophthalmology              |21                |Medium Workload|
|otolaryngologists          |21                |Medium Workload|
|internists                 |21                |Medium Workload|
|emergency medicine         |20                |Medium Workload|
|pulmonologists             |20                |Medium Workload|
|geriatrician               |18                |Medium Workload|
|obstetric anesthesiologists|17                |Medium Workload|
|neurology                  |16                |Lo

## Find the most common procedure

In [0]:
procedures_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/medical_procedures")

In [0]:
from pyspark.sql.functions import count

common_procedure = (
    procedures_df
    .groupBy("procedure_name")
    .agg(count("*").alias("procedure_count"))
    .orderBy("procedure_count", ascending=False)
)

common_procedure.show()


+--------------------+---------------+
|      procedure_name|procedure_count|
+--------------------+---------------+
|X-rays, Ct Scans,...|             26|
|Coronary Artery B...|             25|
|Comprehensive Ger...|             24|
|Interventional Ra...|             23|
|Laser Therapy For...|             22|
|Sedation For Mino...|             22|
|     General Surgery|             22|
|Dermatologic Surgery|             22|
|    Cataract Surgery|             21|
|Intensive Care Ma...|             21|
|         Colonoscopy|             21|
|     Allergy Testing|             21|
|           Endoscopy|             21|
|Advanced Cardiac ...|             21|
|Immunotherapy (al...|             20|
|        Liver Biopsy|             20|
|         Rhinoplasty|             19|
|   Surgical Oncology|             19|
|Medication Manage...|             19|
|Hormone Replaceme...|             18|
+--------------------+---------------+
only showing top 20 rows



##Rank by specialization 

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, sum

# Base: doctors with career revenue (before joining everything else)
doctor_revenue_base = total_revenue_by_doctor_year.groupBy("doctor_id").agg(
    sum("total_yearly_revenue").alias("total_revenue")
).join(
    doctors_df.select("doctor_id", "doctor_name", "specialization"),
    "doctor_id",
    "inner"
)

# Define ranking window by specialization
specialization_rank_window = Window.partitionBy("specialization").orderBy(col("total_revenue").desc())

# Add rank column
doctor_performance_df = doctor_revenue_base.withColumn(
    "specialization_rank",
    rank().over(specialization_rank_window)
)

# Preview top 20
doctor_performance_df.limit(20).display()


doctor_id,total_revenue,doctor_name,specialization,specialization_rank
975,2173939.0,Hettie,allergists,1
619,1987601.0,Shannah,allergists,2
430,1691230.0,Eolanda,allergists,3
742,908837.0,Gui,allergists,4
417,769686.0,Molli,allergists,5
551,672878.0,Morganica,allergists,6
944,555759.0,Elena,allergists,7
101,545986.0,Mireielle,allergists,8
275,408017.0,Danny,allergists,9
925,366652.0,Hyacinthe,allergists,10


In [0]:
from pyspark.sql.functions import coalesce, lit, sum, current_timestamp

# Start with doctor-level base info
final_summary_df = doctors_df.select(
    "doctor_id",
    "doctor_name",
    "specialization",
    "contact_info"
)

# Skip doctor_total_appointments join since you don’t need it

# Join with revenue by year (career total revenue)
final_summary_df = final_summary_df.join(
    total_revenue_by_doctor_year.groupBy("doctor_id")
    .agg(sum("total_yearly_revenue").alias("total_revenue")),
    "doctor_id",
    "left"
).withColumn("total_revenue", coalesce(col("total_revenue"), lit(0.0)))

# Join with specialization demand classification
final_summary_df = final_summary_df.join(
    specialization_demand_yearly.select(
        "specialization",
        "specialization_demand_level"
    ),
    "specialization",
    "left"
).withColumn("specialization_demand_level", coalesce(col("specialization_demand_level"), lit("Unknown")))

# Join with specialization workload classification
final_summary_df = final_summary_df.join(
    specialization_workload_classified.select(
        "specialization",
        col("total_appointments").alias("specialization_total_appointments"),
        "workload_level"
    ),
    "specialization",
    "left"
).withColumn("specialization_total_appointments", coalesce(col("specialization_total_appointments"), lit(0))) \
 .withColumn("workload_level", coalesce(col("workload_level"), lit("Unknown")))

# Add lineage timestamp
final_summary_df = final_summary_df.withColumn("ingestion_date", current_timestamp())


In [0]:
%sql
drop table if exists healthcare_analysis.specialization_summary

In [0]:
final_summary_df.write.mode("overwrite").format("delta").saveAsTable("healthcare_analysis.specialization_summary")

In [0]:
final_summary_df.write.mode("overwrite").parquet(f"abfss://destination@projectadf1.dfs.core.windows.net/specialization_summary")

In [0]:
%sql
SELECT * FROM healthcare_analysis.specialization_summary;

specialization,doctor_id,doctor_name,contact_info,total_revenue,specialization_demand_level,specialization_total_appointments,workload_level,ingestion_date
psychiatry,463,Edith,.@yopmail.com,1780068.0,Low Demand,12,Low Workload,2025-08-25T10:18:01.501Z
surgery,243,Edith,.@yopmail.com,3436703.0,Medium Demand,14,Low Workload,2025-08-25T10:18:01.501Z
otolaryngologists,540,Talya,.@yopmail.com,0.0,Medium Demand,21,Medium Workload,2025-08-25T10:18:01.501Z
infectious disease,858,Jolyn,.@yopmail.com,0.0,High Demand,25,High Workload,2025-08-25T10:18:01.501Z
psychiatry,897,Vinita,.@yopmail.com,0.0,Low Demand,12,Low Workload,2025-08-25T10:18:01.501Z
oncologist,516,Corene,.@yopmail.com,0.0,High Demand,24,High Workload,2025-08-25T10:18:01.501Z
allergists,451,Steffane,.@yopmail.com,0.0,Low Demand,16,Low Workload,2025-08-25T10:18:01.501Z
internists,580,Lynnea,.@yopmail.com,834252.0,Medium Demand,21,Medium Workload,2025-08-25T10:18:01.501Z
dermatology,808,Max,.@yopmail.com,0.0,High Demand,23,High Workload,2025-08-25T10:18:01.501Z
ophthalmology,879,Micheline,.@yopmail.com,185051.0,Medium Demand,21,Medium Workload,2025-08-25T10:18:01.501Z


In [0]:
final_summary_df.printSchema()

root
 |-- specialization: string (nullable = true)
 |-- doctor_id: integer (nullable = true)
 |-- doctor_name: string (nullable = true)
 |-- contact_info: string (nullable = true)
 |-- total_revenue: double (nullable = false)
 |-- specialization_demand_level: string (nullable = false)
 |-- specialization_total_appointments: long (nullable = false)
 |-- workload_level: string (nullable = false)
 |-- ingestion_date: timestamp (nullable = false)

